In [1]:
import pandas as pd
import dataframeLoader as dfl
import plotly

root = '.dataDir'
## load timeseries data from CSV files
df10 = dfl.loadStrucDataFromFileRegex(root, 'STRUCTURED-*.csv')
# metricsArr = ['cpu_used', 'download_workers_count', 'memory_used', 'task_queue_length', 'infra_access_latency', 'pod_cpu_usage', 'pod_memory_usage']    
metricsArr = ['cpu_used','task_queue_length', 'memory_used']
df = dfl.loadPrometheusDataFromFileRegex(root, 'securiti_appliance_', metricsArr, '.csv')
df9 = dfl.loadUnstrucDataFromFileRegex(root, 'UNSTRUCTURED-*.csv')
df = pd.concat([df, df9, df10], ignore_index=True)

display(df)

In [ ]:
## Generate top performing appliances between time-period.
fromDt = '2024-08-05'
toDt = '2024-09-09'
df10 = dfl.loadDataFrameFromFileRegex(root, 'UNSTRUCTURED-*.csv', metrics='dataScanned')
df10['ts']=pd.to_datetime(df10['ts'],unit='ms')
df10 = df10[(df10['ts'] >= fromDt) & (df10['ts'] <= toDt) ]
df10 = df10.groupby(['tenant', 'pod']).agg(dataScanned=('dataScannedInGB', 'sum')).reset_index()
df10 = df10.sort_values('dataScanned', ascending=False)
display(df10.head(20))


In [15]:
## Generate timeseries fascet graph for an appliance_id
from importlib import reload 
import dataframeLoader as dfl
import plotly.io as pio
pio.renderers.default = 'browser'
reload(dfl)

cat_order = [
            "dataScannedinGB",
            "numberOfColsScanned", "numberOfChunksScanned", 
             "scanTime", "uniqPodCount",
             "numFilesScanned", "avgFileSizeInMB", "IdleTimeInHrs",
             "cpu_used_avg", "memory_used_avg",
             "tmp_taskq_avg", "taskq_max"
             ]
appliance_id='58e98e10-1b19-4c84-93c0-db2ad5903b80'
from_dt='2024-08-15'
to_dt = '2024-10-15'
title = 'Appliance plot for appliance_id '+appliance_id+' between '+from_dt+' and '+to_dt
dfp = df[(df['appliance_id'] == appliance_id) 
         & (df['metrics'].isin(cat_order))
         & ((df['ts'] >= from_dt) & (df['ts'] <= to_dt))
         ].drop_duplicates()
fig  = dfl.plotMetricsFacetForApplianceId(dfp, title, {"metrics": cat_order})
# fig.update_yaxes(type="log")
fig.update_layout(plot_bgcolor="black")
fig.show()